<a href="https://colab.research.google.com/github/bdemchak/cytoscape-jupyter/blob/main/gangsu/basic%20protocol%201.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a reproduction of the [Biological Network Exploration with Cytoscape 3](https://pubmed.ncbi.nlm.nih.gov/25199793/) Basic Protocol 1, which loads an s. cervesiae network, filters out unneeded nodes, lays out the resulting network, creates clusters of similar nodes and then performs an enrichment calculation on one cluster.

Note that this workflow executes in a Jupyter Notebook running on a cloud server (e.g., Google Colab) and communicates with a copy of Cytoscape running on your workstation. For version of this workflow that runs on the same workstation as Cytoscape, see [here](https://github.com/cytoscape/py4cytoscape/tree/master/tests).

---
# Setup data files, py4cytoscape and Cytoscape connection
**NOTE: To run this notebook, you must manually start Cytoscape first -- don't proceed until you have started Cytoscape.**

This workflow requires two files that are located in cloud storage:

* BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab (network file)
* GDS112_full.soft (annotation file)

Both files reside in a Dropbox folder, and they are downloaded by this workflow as needed.

## Setup: Fetch latest py4cytoscape




**Note that you can fetch from the latest Github unreleased version by setting _PY4CYTOSCAPE to 'git+https://github.com/cytoscape/py4cytoscape' immediately before the exec() call. To fetch a particular branch, add '@' to the end (e.g., 'git+https://github.com/cytoscape/py4cytoscape@0.0.12').**

To load the default (PyPI) py4cytoscape version, do not set _PY4CYTOSCAPE at all.

In [1]:
#_PY4CYTOSCAPE = 'git+https://github.com/cytoscape/py4cytoscape@1.6.0' # optional
import requests

exec(requests.get("https://raw.githubusercontent.com/cytoscape/jupyter-bridge/master/client/p4c_init.py").text)

IPython.display.Javascript(_PY4CYTOSCAPE_BROWSER_CLIENT_JS) # Start browser client

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

Collecting py4cytoscape

Downloading py4cytoscape-1.6.0-py3-none-any.whl (176 kB)

Output()

Collecting backoff

...

Installing collected packages: texttable, colour, igraph, colorbrewer, backoff, python-igraph, py4cytoscape

Successfully installed backoff-2.2.1 colorbrewer-0.2.0 colour-0.1.5 igraph-0.10.3 py4cytoscape-1.6.0 python-igraph-0.10.3 texttable-1.6.7

NumExpr defaulting to 2 threads.

Loading Javascript client ... 2c2eb1da-6058-4292-8bc2-531f8da8aaf5 on https://jupyter-bridge.cytoscape.org
ADVICE: WHEN RUNNING UNDER COLAB, DO NOT RE-RUN THIS CELL WITHOUT MANUALLY EXECUTING Runtime | Factory Reset Runtime FROM THE COLAB MENU FIRST.


<IPython.core.display.Javascript object>

## Setup: Sanity test to verify Cytoscape connection

By now, the connection to Cytoscape should be up and available. To verify this, try a simple operation that doesn't alter the state of Cytoscape.

In [2]:
p4c.cytoscape_version_info()


{'apiVersion': 'v1',
 'cytoscapeVersion': '3.10.0-SNAPSHOT',
 'automationAPIVersion': '1.6.0',
 'py4cytoscapeVersion': '1.6.0',
 'jupyterBridgeVersion': '0.0.2'}

## Setup: Notebook data files
Create the 'output' directory, which will be used to store files uploaded from Cytoscape.

This is a good place to prepare any other system resources that might be needed by downstream Notebook cells.





In [3]:
!rm -r output/
!ls -l 
OUTPUT_DIR = 'output/'

rm: cannot remove 'output/': No such file or directory
total 8
drwxr-xr-x 2 root root 4096 Jan 14 20:50 logs
drwxr-xr-x 1 root root 4096 Jan  9 14:36 sample_data


## Setup: Import source data files
The network and annotation files are in a Dropbox folder, and this cell downloads them into the default Sandbox from where Cytoscape will access them.

The files could just as well have been on any cloud resource, including Google Drive, Github, Microsoft OneDrive or a private web site. Note that in this case, the network file was so large that it could not be saved on GitHub, so Dropbox was a handy alternative.

*An alternative would be to load the files into this Notebook's file system (or create them there) and then download those files to the Sandbox. Loading them into the Notebook file system would require the use of Notebook "!" commands (e.g., !wget).*


**Sandboxing is explained in https://py4cytoscape.readthedocs.io/en/latest/concepts.html#sandboxing**

In [4]:
p4c.sandbox_set(None) # Revert to default sandbox in case some other workflow selected a different one

res_mitab = p4c.sandbox_url_to("https://www.dropbox.com/s/8wc8o897tsxewt1/BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab?dl=0", "BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab")
print(f'Network file BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab has {res_mitab["fileByteCount"]} bytes')

Network file BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab has 166981992 bytes


---
# Load the Protein-protein Interaction Network into Cytoscape
The network is contained in the s. cerevisiae MITAB file.

Note that in this cell, the `import_network_from_file()` function (incorrectly) throws an exception in pre-3.10.0 Cytoscape. To ignore the exception, we enclose it in a try/except block.

**Note:** Once the CYTOSCAPE-12772 issue is solved, we can remove the try/except block.


In [5]:
from requests import HTTPError
p4c.close_session(False)

try:
  p4c.import_network_from_file('BIOGRID-ORGANISM-Saccharomyces_cerevisiae-3.2.105.mitab')
except:  
  pass
if p4c.get_network_count() != 1:
  raise Exception('Failed to load network')
net_suid = p4c.get_network_suid()
print(f'Network identifier: {net_suid}')



Network identifier: 501


---
# Import the gene expression data
The expression data is downloaded and merged into the network's node attribute table. 

The `Gene ID` column in the expression data is used as the key matched to the node attribute's `shared name` column, and the first 82 lines in the expression data is skipped because it doesn't contain data.

---
*Tip:* This cell shows how to create code that works around changes in Cytoscape capabilities. 

In this case, starting with Cytoscape 3.9.0, the `load_table_data_from_file()` function works as expected, so the gene expression data is merged into the node attribute table. 

Prior to 3.9.0, `load_table_data_from_file()` didn't work. As a workaround, we do most of the work in Pandas and then import the dataframe into the node attribute table. After Pandas reads the CSV, we will try to match dataframe Gene ID column to the `name` column in the Cytoscape node attribute table. To do this, we must explicitly set the Gene ID as a string (even though it's originally parsed as a number) because Cytoscape's `name` column is already a string. 



In [6]:
if p4c.check_supported_versions(cytoscape='3.9') is None:
  # Load file directly into Sandbox so Cytoscape can import it
  res_soft = p4c.sandbox_url_to("https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0", "GDS112_full.soft")
  print(f'Annotation file GDS112_full.soft has {res_soft["fileByteCount"]} bytes')

  res = p4c.load_table_data_from_file('GDS112_full.soft', start_load_row=83, data_key_column_index=10, delimiters='\t')
  print(f'Load result contains table identifiers: {res["mappedTables"]}')
else:
  # Load file into Notebook file system so Python can import it, tweak it, and download to Cytoscape
  !wget -q --no-check-certificate https://www.dropbox.com/s/r15azh0xb53smu1/GDS112_full.soft?dl=0
  !mv GDS112_full.soft?dl=0 GDS112_full.soft

  import pandas as df
  GDS112_full = df.read_csv('GDS112_full.soft', skiprows=82, sep='\t')
  GDS112_full.dropna(subset=['Gene ID'], inplace=True)
  GDS112_full['Gene ID'] = df.to_numeric(GDS112_full['Gene ID'], downcast='integer')
  GDS112_full = GDS112_full.astype({'Gene ID': 'string'})
  print(GDS112_full.dtypes)
  print(GDS112_full)
  p4c.load_table_data(GDS112_full, data_key_column='Gene ID')

  import os
  os.remove('GDS112_full.soft')


Annotation file GDS112_full.soft has 5536880 bytes
Load result contains table identifiers: [472, 510]


---
# Filter the Network with the Genes that have Expression Data
For this, we assume that if a node has no *Gene symbol*, it also has no expression data. 

The filter compares each node's *Gene symbol* attribute to a regular expression. If there is a match, the gene is selected; for no match, the gene isn't selected.

In [7]:
res = p4c.create_column_filter('SymbolOK', 'Gene symbol', '[A-Z0-9]*', 'REGEX')
print(f'Nodes selected: {len(res["nodes"])}')

No edges selected.
Nodes selected: 5501


---
# Create a New Network with the Selected Subset
Create a subnetwork containing only nodes selected by the filter (i.e., having a *Gene symbol* value, which implies that expression data is present for that node).

This could take several minutes.

At the end, you should see a view containing all nodes laid out. 

If you see only a single rectangle, it could be that your Cytoscape is set to operate with a small stack size. To increase the stack:

1. terminate Cytoscape

2. a) upgrade Cytoscape to 3.9.0 or later 

  ... or b) use a text editor to add -Xss5M to the cytoscape.vmoptions file in your Cytoscape program directory

3. restart Cytoscape

4. re-run this workflow

In [8]:
new_suid = p4c.create_subnetwork()
print(f'New network identifier: {new_suid}')

New network identifier: 1388406


## Get rid of the original network, which isn't needed anymore

In [9]:
p4c.delete_network(net_suid)
net_suid = new_suid

---
# Identify Network Modules
The overall strategy is to find clusters of nodes that share some common attribute. In this case, we use expression data values. Specifically:

* Load Cytoscape's clusterMaker2 app
* Use clusterMaker2 to create a dendogram showing a hierarchy of similar network modules


## Install clusterMaker2 if it hasn't already been installed

In [10]:
p4c.install_app('clusterMaker2')

{}


{}

## Identify network modules
Create a hierarchic clustering of similar nodes based on the expression data columns. Cytoscape renders the hierarchy as a dendogram.

*Tip:* Cytoscape's dendogram window can be used to manually explore module similarity. 

In [11]:
dendo_clustering = p4c.commands_post('cluster hierarchical showUI=true clusterAttributes=false nodeAttributeList="GSM1029,GSM1030,GSM1032,GSM1033,GSM1034"')

# dendo_clustering is a dictionary [{nodeOrder: [{nodeName: xxx, suid: sss}, ...]}
#                                   {nodeTree: [{name: ggg, left: lll, right: rrr}]}]
# where nodeOrder is a mapping between a leaf node name xxx and the suid sss of a network node,
# and nodeTree is a tree where the left node lll and right node rrr can be leaf nodes xxx or
# internal nodes ggg. 

---
# Perform an enrichment analysis using the gprofiler package

Use a package commonly available in PyPI to calculate functional enrichment for nodes similar to a node in which we may be interested. It's an example of how Cytoscape can work together with Python-based libraries to achieve a useful result.

In this case, we choose HBT1 (entrez-gene ID 851303).

1. Find SUID of network's HBT1 node

1. Find a set of nodes similar to HBT1 by collecting nodes nearby in the tree

1. Use each node's SUID to look up its entrez-gene ID

1. Pass the set of entrez-gene IDs to gprofiler as an enrichment query




## Find HBD1 in the similarity tree

In [12]:
node_suid = p4c.node_name_to_node_suid('851303')[0] # Use entrez-gene ID to get SUID for HBT1

## Collect set of SUIDs representing 85 similar nodes
Note that we use custom functions to parse and traverse the dendogram's similarity tree.

We use *wget* to copy the functions into a local directory so we can them import them and then call them.

In [13]:
!wget -c https://raw.githubusercontent.com/cytoscape/py4cytoscape/master/tests/Notebooks/parse_dendogram.py
import parse_dendogram as pde # Use custom functions to decode dendogram tree

node_order = dendo_clustering[0]['nodeOrder']
node_tree = dendo_clustering[0]['nodeTree']

node_bag = pde.create_node_bag(node_order, node_tree)
similar_nodes = list(pde.find_node_set(node_suid, 85, node_order, node_bag))

--2023-01-14 21:01:56--  https://raw.githubusercontent.com/cytoscape/py4cytoscape/master/tests/Notebooks/parse_dendogram.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1852 (1.8K) [text/plain]
Saving to: ‘parse_dendogram.py’

parse_dendogram.py  100%[===================>]   1.81K  --.-KB/s    in 0s      

2023-01-14 21:01:56 (26.1 MB/s) - ‘parse_dendogram.py’ saved [1852/1852]



## Using SUIDs, query Cytoscape for each node's entrez-gene ID

In [14]:
suid_to_entrez_gene = p4c.get_table_columns(columns='name')['name']
entrez_gene_query = [int(suid_to_entrez_gene[suid])  for suid in similar_nodes]

print(entrez_gene_query)

[851596, 854170, 851747, 852389, 856016, 855144, 852088, 851934, 856749, 853159, 856866, 851380, 851474, 853102, 852032, 856671, 853338, 852664, 852763, 855308, 855749, 854357, 853758, 856541, 856925, 851468, 855562, 851013, 851625, 852592, 851419, 850711, 856913, 854337, 855757, 851084, 856360, 855259, 852771, 850840, 853238, 854114, 851582, 851980, 850928, 852064, 854263, 855126, 850495, 856417, 853920, 853536, 856492, 856536, 854856, 853100, 854109, 855887, 850295, 855363, 853614, 854123, 853766, 854152, 855878, 851303, 854738, 856313, 854125, 854000, 851541, 854085, 850354, 856616, 853693, 854261, 854206, 850598, 853405, 856543, 851141, 851100, 851649, 853613, 853281]


## Install gprofiler package if it's not already installed

In [15]:
!pip install gprofiler-official
from gprofiler import GProfiler

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Use entrez-gene IDs to query gprofiler for GO functional enrichment

In [16]:
gp = GProfiler(user_agent='py4cytoscape', return_dataframe=True)
gp.profile(organism='scerevisiae', query=entrez_gene_query)

,source,native,name,p_value,significant,description,term_size,query_size,intersection_size,effective_domain_size,precision,recall,query,parents
0,GO:MF,GO:0004679,AMP-activated protein kinase activity,0.000005,True,"""Catalysis of the reaction: ATP + a protein = ...",8,84,5,6549,0.059524,0.625000,query_1,[GO:0004674]
1,GO:CC,GO:0031588,nucleotide-activated protein kinase complex,0.000007,True,"""A protein complex that possesses nucleotide-d...",9,84,5,6566,0.059524,0.555556,query_1,"[GO:0140535, GO:1902911]"
2,GO:CC,GO:0016514,SWI/SNF complex,0.000161,True,"""A SWI/SNF-type complex that contains 8 to 14 ...",15,84,5,6566,0.059524,0.333333,query_1,[GO:0070603]
3,GO:BP,GO:0070784,regulation of growth of unicellular organism a...,0.000293,True,"""Any process that modulates the frequency, rat...",38,84,7,6539,0.083333,0.184211,query_1,"[GO:0070783, GO:1900428]"
4,GO:BP,GO:1900428,regulation of filamentous growth of a populati...,0.000601,True,"""Any process that modulates the frequency, rat...",42,84,7,6539,0.083333,0.166667,query_1,"[GO:0010570, GO:0044182]"
5,GO:CC,GO:0005576,extracellular region,0.000822,True,"""The space external to the outermost structure...",127,84,10,6566,0.119048,0.078740,query_1,[GO:0110165]
6,GO:BP,GO:0010570,regulation of filamentous growth,0.000836,True,"""Any process that modulates the frequency, rat...",44,84,7,6539,0.083333,0.159091,query_1,"[GO:0030447, GO:0040008]"
7,GO:CC,GO:0009277,fungal-type cell wall,0.002098,True,"""A rigid yet dynamic structure surrounding the...",141,84,10,6566,0.119048,0.070922,query_1,[GO:0005618]
8,GO:BP,GO:0070786,positive regulation of growth of unicellular o...,0.002237,True,"""Any process that activates or increases the f...",19,84,5,6539,0.059524,0.263158,query_1,"[GO:0070783, GO:0070784, GO:1900430]"
9,GO:CC,GO:0005618,cell wall,0.003032,True,"""The rigid or semi-rigid envelope lying outsid...",147,84,10,6566,0.119048,0.068027,query_1,[GO:0030312]
